In [ ]:
import pandas as pd
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

default_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


#PLEASE NOTICE
#the following values are hardcoded and they must be edited before running
#the program in any new environment
DATASET_PATH = '/content/Datasets/'
DATAFRAMES_PATH = '/content/drive/MyDrive/Colab_Notebooks/Dataframes/'

if not Path(DATASET_PATH).exists():
    raise FileNotFoundError(
        'DATASET_PATH is hardcoded, please adjust to point to gsv_cities')

if not Path(DATAFRAMES_PATH).exists():
    raise FileNotFoundError(
        'DATAFRAMES_PATH is hardcoded, please adjust to point to gsv_cities')

class GSVCitiesDataset(Dataset):
    def __init__(self,
                 cities=['London', 'Boston'],
                 img_per_place=4,
                 min_img_per_place=4,
                 random_sample_from_each_place=True,
                 transform=default_transform,
                 dataframes_path=DATAFRAMES_PATH,
                 dataset_path=DATASET_PATH
                 ):
        super(GSVCitiesDataset, self).__init__()
        self.dataframes_path = dataframes_path
        self.dataset_path = dataset_path
        self.cities = cities

        assert img_per_place <= min_img_per_place, \
            f"img_per_place should be less than {min_img_per_place}"
        self.img_per_place = img_per_place
        self.min_img_per_place = min_img_per_place
        self.random_sample_from_each_place = random_sample_from_each_place
        self.transform = transform

        # generate the dataframe contraining images metadata
        self.dataframe = self.__getdataframes()

        # get all unique place ids
        self.places_ids = pd.unique(self.dataframe.index)
        self.total_nb_images = len(self.dataframe)

    def __getdataframes(self):
        '''
            Return one dataframe containing
            all info about the images from all cities

            This requieres DataFrame files to be in a folder
            named Dataframes, containing a DataFrame
            for each city in self.cities
        '''
        # read the first city dataframe
        df = pd.read_csv(self.dataframes_path+f'{self.cities[0]}.csv')
        df = df.sample(frac=1)  # shuffle the city dataframe


        # append other cities one by one
        for i in range(1, len(self.cities)):
            tmp_df = pd.read_csv(
                self.dataframes_path+f'{self.cities[i]}.csv')

            # Now we add a prefix to place_id, so that we
            # don't confuse, say, place number 13 of NewYork
            # with place number 13 of London ==> (0000013 and 0500013)
            # We suppose that there is no city with more than
            # 99999 images and there won't be more than 99 cities
            # TODO: rename the dataset and hardcode these prefixes
            prefix = i
            tmp_df['Id'] = tmp_df['Id'] + (prefix * 10**5)
            tmp_df = tmp_df.sample(frac=1)  # shuffle the city dataframe

            df = pd.concat([df, tmp_df], ignore_index=True)

        # keep only places depicted by at least min_img_per_place images
        res = df[df.groupby('Id')['Id'].transform(
            'size') >= self.min_img_per_place]
        return res.set_index('Id')

    def __getitem__(self, index):
        place_id = self.places_ids[index]

        # get the place in form of a dataframe (each row corresponds to one image)
        place = self.dataframe.loc[place_id]

        # sample K images (rows) from this place
        # we can either sort and take the most recent k images
        # or randomly sample them
        place = place.sample(n=self.img_per_place)

        imgs = []
        for i, row in place.iterrows():
            img_path = self.dataset_path + row["Path"]
            img = self.image_loader(img_path)

            if self.transform is not None:
                img = self.transform(img)

            imgs.append(img)

        # NOTE: contrary to image classification where __getitem__ returns only one image
        # in GSVCities, we return a place, which is a Tesor of K images (K=self.img_per_place)
        # this will return a Tensor of shape [K, channels, height, width]. This needs to be taken into account
        # in the Dataloader (which will yield batches of shape [BS, K, channels, height, width])
        return torch.stack(imgs), torch.tensor(place_id).repeat(self.img_per_place)

    def __len__(self):
        '''Denotes the total number of places (not images)'''
        return len(self.places_ids)

    @staticmethod
    def image_loader(path):
        return Image.open(path).convert('RGB')